##Singapore


In [ ]:
#read singapore listing dataset
import pandas as pd

df_sing_list = pd.read_csv('/content/listings.csv')

df_sing_review = pd.read_csv("/content/reviews.csv")

In [ ]:
df_sing_list

In [ ]:
df_sing_list['id'].nunique()

In [ ]:
df_sing_list.info()

In [ ]:
df_sing_list.columns

In [ ]:
#need to clarify which columns of minimun night we needed
df_sing_list[['minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights']]

In [ ]:
df_sing_list['price'] = df_sing_list['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

def classify_price(price):
    try:
        price = float(price)
        if 1<= price < 84:
            return 'Low'
        elif 84 <= price < 157:
            return 'Medium'
        elif 157 <= price < 271:
            return 'Medium High'
        elif 271 <= price < 13000:
            return 'High'
    except:
        return 'Invalid'
df_sing_list['price_range'] = df_sing_list['price'].apply(classify_price)

df_sing_list['price_range']

In [ ]:
#Define the price range of room
def classify_price(price):
    try:
        price = float(price)
        if 1<= price < 84:
            return 'Low'
        elif 84 <= price < 157:
            return 'Medium'
        elif 157 <= price < 271:
            return 'Medium High'
        elif 271 <= price < 13000:
            return 'High'
    except:
        return 'Invalid'
df_sing_list['price_range'] = df_sing_list['price'].apply(classify_price)

df_sing_list['price_range']

In [ ]:
df_sing_list['price_range'].value_counts()

In [ ]:
df_sing_list.info()

In [ ]:
#demand ratio between each price range
df_sing_list.groupby('price_range')['estimated_occupancy_l365d'].sum()

In [ ]:
#supply ratio between each price range
df_sing_list.groupby('price_range')['availability_365'].sum()

## 2. Pricing Optimization Relationships

In [ ]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [ ]:
df_sing_list.info()

In [ ]:
df_sing_list

In [ ]:
#drop the null value in price column
df_sing_list.dropna(subset=['price'],inplace = True)


In [ ]:
#price range distribution for each region
df_sing_list.groupby('neighbourhood_group_cleansed')['price_range'].value_counts()

In [ ]:
#supply count for each region in specific
df_sing_list.groupby('neighbourhood_group_cleansed')['neighbourhood_cleansed'].value_counts()

In [ ]:
df_sing_list['neighbourhood_group_cleansed'].value_counts()

In [ ]:
#demand of region
df_sing_list.groupby(['neighbourhood_group_cleansed','neighbourhood_cleansed'])['estimated_occupancy_l365d'].sum().sort_values(ascending = False)

In [ ]:
#demand rate of neighbourhood group
df_sing_list.groupby(['neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed'])['estimated_occupancy_l365d'].sum().sort_values(ascending= False)

In [ ]:
#Price variation by room type,supply
df_sing_list.groupby('room_type')['price_range'].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count values
counts = df_sing_list.groupby('room_type')['price_range'].value_counts().unstack().fillna(0)


# Plot
counts.plot(kind='bar', stacked=True)
plt.title('Price Range Distribution by Room Type')
plt.xlabel('Room Type')
plt.ylabel('Count')
plt.legend(title='Price Range')
plt.tight_layout()
plt.xticks(rotation=0)
plt.show()


In [ ]:
#read csv with tourist spot information
df_tourist_spot = pd.read_csv('/content/reverse_geocode_with_district_mrt_tourist_full_sample3659.csv')
df_tourist_spots = df_tourist_spot['tourist_spot_distance_bin']

In [ ]:
#read csv with distance to MRT inforamtion
df_exact_spot = pd.read_csv('/content/reverse_geocode_with_district_mrt_tourist_exact_bins_sample3659.csv')
df_exact_spot.info(verbose=True, show_counts=True)

In [ ]:
# add distance to MRT in df_sing_list
df_sing_list['distance_to_nearest_MRT_km']= df_exact_spot['distance_to_nearest_MRT_km']
df_sing_list['distance_to_nearest_MRT_km']

In [ ]:
df_exact_spot['distance_to_nearest_tourist_spot_km']

In [ ]:
df_sing_list['tourist_spot'] = df_tourist_spots
df_sing_list

In [ ]:
df_sing_list.groupby('room_type')['price_range'].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Updated data from your output
data = {
    'room_type': ['Entire home/apt'] * 4 + ['Hotel room'] * 4 + ['Private room'] * 4 + ['Shared room'] * 3,
    'price_range': ['High', 'Medium High', 'Medium', 'Low',
                    'Medium', 'Medium High', 'High', 'Low',
                    'Low', 'Medium', 'Medium High', 'High',
                    'Low', 'High', 'Medium'],
    'count': [602, 507, 170, 10,
              15, 9, 4, 2,
              613, 482, 150, 48,
              36, 9, 4]
}

# Create DataFrame
df_grouped = pd.DataFrame(data)

# Set desired order for room types and price ranges
room_order = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
price_order = ['Low', 'Medium', 'Medium High', 'High']
df_grouped['room_type'] = pd.Categorical(df_grouped['room_type'], categories=room_order, ordered=True)
df_grouped['price_range'] = pd.Categorical(df_grouped['price_range'], categories=price_order, ordered=True)

# Pivot the data for grouped bar chart
pivot_df = df_grouped.pivot_table(index='room_type', columns='price_range', values='count', aggfunc='sum').fillna(0)

# Plot grouped bar chart using Matplotlib
bar_width = 0.2
x = np.arange(len(pivot_df.index))

fig, ax = plt.subplots(figsize=(10, 6))

for i, price_range in enumerate(price_order):
    ax.bar(x + i * bar_width, pivot_df[price_range], width=bar_width, label=price_range)

# Formatting
ax.set_xticks(x + bar_width * (len(price_order) - 1) / 2)
ax.set_xticklabels(pivot_df.index)
ax.set_xlabel('Room Type')
ax.set_ylabel('Count')
ax.set_title('Distribution of Price Ranges Across Room Types')
ax.legend(title='Price Range')
plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px

grouped = df_sing_list.groupby('room_type')['price_range'].value_counts().reset_index(name='count')

# Create bar chart
fig = px.bar(
    grouped,
    x='room_type',
    y='count',
    color='price_range',
    barmode='group',
    title='Distribution of Price Ranges Across Room Types',
    labels={'room_type': 'Room Type', 'count': 'Count', 'price_range': 'Price Range'}
)

fig.show()


In [ ]:
#Price range between different room type
import matplotlib.pyplot as plt

# Group and count
grouped = df_sing_list.groupby(['room_type', 'price_range']).size().reset_index(name='count')

# Get unique room types
room_types = grouped['room_type'].unique()

# Create separate bar charts for each room type
for room in room_types:
    subset = grouped[grouped['room_type'] == room]

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.bar(subset['price_range'], subset['count'], color='orange')
    plt.title(f"Price Range Distribution for {room}")
    plt.xlabel('Price Range')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
#plot pie chart for price range distribution for different room type

import matplotlib.pyplot as plt

# Group and count
grouped = df_sing_list.groupby(['room_type', 'price_range']).size().reset_index(name='count')

# Get unique room types
room_types = grouped['room_type'].unique()

# Custom colors
CUSTOM_COLORS = ['#d62728','#1f77b4', '#ff7f0e', '#2ca02c']

# Create separate pie charts for each room type
for room in room_types:
    subset = grouped[grouped['room_type'] == room]

    values = subset['count'].values
    labels = subset['price_range'].astype(str).values  # ensure labels are strings

    # Cycle through colors if slices > len(CUSTOM_COLORS)
    colors = [CUSTOM_COLORS[i % len(CUSTOM_COLORS)] for i in range(len(values))]

    plt.figure(figsize=(8, 8))
    wedges, texts, autotexts = plt.pie(
        values,
        labels=labels,
        colors=colors,               # Apply custom colors
        autopct='%1.1f%%',           # Show percentages
        startangle=100,              # Rotate so first wedge starts at the top
        counterclock=False,          # Clockwise for readability
        wedgeprops=dict(edgecolor='white')
    )
    plt.title(f"Price Range Distribution for {room}", pad=20)
    plt.axis('equal')  # Make it a circle
    plt.tight_layout()
    plt.show()

In [ ]:
result = df_sing_list.groupby('tourist_spot')['price_range'].value_counts()
# Filter for 'High' price_range in distance of tourist spot
high_prices = result[result.index.get_level_values('price_range') == 'High']
high_prices

In [ ]:
high_df= df_sing_list[df_sing_list['price_range']=='High']
high_df['price_range']

In [ ]:
# Filter for 'Medium High' price_range to distance of tourist spot
Medium_high_prices = result[result.index.get_level_values('price_range') == 'Medium High']
Medium_high_prices

In [ ]:
# Filter for 'Medium ' price_range
Medium_prices = result[result.index.get_level_values('price_range') == 'Medium']
Medium_prices

In [ ]:
# Filter for 'low' price_range
low_prices = result[result.index.get_level_values('price_range') == 'Low']
low_prices

In [ ]:
df_sing_list['room_type'].value_counts()

In [ ]:
#room type distribution chart
Room_type = df_sing_list['room_type'].value_counts()

colors = ['#1f77b4', 'skyblue', 'orange', 'green']

Room_type.plot(kind='bar',color=colors, figsize=(8, 6))
plt.xticks(rotation=0)
plt.title('Room type distribution')


In [ ]:
#Pricing analysis (price ↔ occupancy)
import pandas as pd
import matplotlib.pyplot as plt
#plt.ylim(0, 2000)  # Set Y-axis from 0 to 100
plt.scatter(df_sing_list['price'],df_sing_list["estimated_occupancy_l365d"])
plt.title('Price vs occupancy')

In [ ]:
#Pricing vs room type, MRT, tourist spots
room_type_price = df_sing_list.groupby('room_type')['price'].sum()

colors = ['#1f77b4', 'skyblue', 'orange', 'green','purple']

room_type_price.plot(kind='bar',color=colors, figsize=(8, 6))
plt.xticks(rotation=0)
plt.title('Room Type vs Price')

In [ ]:
# bar chart for response rate 90%

import matplotlib.pyplot as plt

# Data
labels = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
values = [0.534, 0.421, 0.796, 0.833]

# Create bar chart
plt.figure(figsize=(8, 5))
plt.bar(labels, values, color = ['#1f77b4', 'skyblue', 'orange', 'green'])

# Add titles and labels
plt.title('90% Response Rate Comparison')
plt.ylabel('Host response rate > 90%')
plt.ylim(0, 1)  # Set y-axis from 0 to 1 for clarity

# Show values on top of bars
for i, v in enumerate(values):
    plt.text(i, v + 0.02, f"{v:.2f}", ha='center')

# Display chart
plt.show()


In [ ]:
#estimated occupancy by room type
import matplotlib.pyplot as plt

# Data for estimated occupancy percentages
room_types = ['Entire home/apt','Private Room', 'Shared Room','Hotel room']
estimated_occupancy = [23.9,20.6,65.3,56.6]  # Percentages from your calculations

# Create the bar graph
plt.figure(figsize=(8, 6))
plt.bar(room_types, estimated_occupancy, color = ['#1f77b4', 'skyblue', 'orange', 'green'])

# Add labels and title
plt.xlabel('Room Type')
plt.ylabel('Estimated Occupancy (%)')
plt.title('Estimated Occupancy by Room Type')
plt.ylim(0, 100)  # Set y-axis range from 0 to 100

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data
categories = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
response_rates = [0.53, 0.42, 0.80, 0.83]

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot bars
bars = ax.bar(categories, response_rates, color=['#1f77b4', 'skyblue', 'orange', 'green'])

# Plot line connecting the tops of the bars
ax.plot(categories, response_rates, color='black', marker='o', linestyle='-')

# Add labels and title
ax.set_ylabel('Host response rate > 90%')
ax.set_title('90% Response Rate Comparison')
ax.set_ylim(0, 1.0)

# Annotate each bar with its value
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height + 0.04, f'{response_rates[i]:.2f}', ha='center')

plt.tight_layout()
plt.show()

In [ ]:
#calculate the HHI rate for each location

import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# --- 1) Example: load your Airbnb data (replace with your path) --- # df = pd.read_csv("airbnb_sg.csv")# Required columns: listing_id, host_id, nearest_tourist_spot assert {"listing_id","host_id","nearest_tourist_spot"}.issubset(df.columns)
# --- 2) Compute HHI per tourist spot ---
def hhi_for_series(host_ids: pd.Series) -> float:
    """HHI on 0–1 scale for a set of host_ids within a spot."""
    shares = host_ids.value_counts(normalize=True)
    return float((shares**2).sum())

hhi_by_spot = (
    df_sing_list.groupby("neighbourhood_cleansed")["host_id"]
      .apply(hhi_for_series)
      .rename("HHI_0to1")
      .reset_index()
)
# Add 0–10,000 scaled version (common in econ reports)
hhi_by_spot["HHI_0to10000"] = (hhi_by_spot["HHI_0to1"] * 10000).round(1)
# Sort (highest concentration first)
hhi_by_spot = hhi_by_spot.sort_values("HHI_0to1", ascending=False)
# --- 3) Plot bar chart (HHI 0–10,000) ---
plt.figure(figsize=(10, 5))
plt.bar(hhi_by_spot["neighbourhood_cleansed"], hhi_by_spot["HHI_0to1"])
plt.title("HHI by Tourist Spot (0–10,000 scale)")
plt.ylabel("HHI (0–10,000)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
# --- 4) (Optional) Add reference lines for interpretation --- # Low <1500, Moderate 1500–2500, High >2500 # Uncomment if you want bands on the same chart: # for y in [1500, 2500]: #     plt.axhline(y=y, linestyle="--")# --- 5) (Optional) Save the table for your dashboard --- # hhi_by_spot.to_csv("hhi_by_tourist_spot.csv", index=False)

In [ ]:
# calculate the host id count for 'Sembawang', 'Central Water Catchment', 'Jurong East'
areas1= ['Sembawang', 'Central Water Catchment', 'Jurong East']
df_monopoly1= df_sing_list[df_sing_list['neighbourhood_cleansed'].isin(areas1)]
df_monopoly1['host_id'].count()

In [ ]:
# calculate the host id count for 'Changi', 'Western Water Catchment', 'Sungei Kadut', 'Marina South'
areas = ['Changi', 'Western Water Catchment', 'Sungei Kadut', 'Marina South']
df_monopoly= df_sing_list[df_sing_list['neighbourhood_cleansed'].isin(areas)]
df_monopoly['host_id'].count()

In [ ]:
df_monopoly['neighbourhood_cleansed'].value_counts()

In [ ]:
#Host id for Marina South
df_sing_list[df_sing_list['neighbourhood_cleansed'] =='Marina South']['host_id']

In [ ]:
#hhi rate
hhi_by_spot["HHI_0to1"]

In [ ]:
#Data cleaning for those price that is incorrected with the website
# Build the mask for your condition
mask = (
    (df_sing_list['room_type'] == 'Shared room') &
    (df_sing_list['price'] == 9999.00)
)

# Update price to 200 for those rows
df_sing_list.loc[mask, 'price'] = 200




In [ ]:
df_sing_list[(df_sing_list['room_type'] == 'Shared room') & (df_sing_list['price'] == 200)][['price','estimated_occupancy_l365d','estimated_revenue_l365d']]

In [ ]:
#confirm with the price that is amended in previous coding
df_sing_list[df_sing_list['room_type'] == 'Shared room']['price']

In [ ]:
#estimated revenue in entire home/apt by different location
entire_home = df_sing_list[df_sing_list['room_type']== 'Entire home/apt']
entire_home.groupby('neighbourhood_cleansed') ['estimated_revenue_l365d'].sum().sort_values(ascending = False)

In [ ]:
#entire home supply by location
entire_home.groupby('neighbourhood_cleansed')['listing_id'].count().sort_values(ascending= False)

In [ ]:
# entire home average price in outram
474576/59

In [ ]:
private_room = df_sing_list[df_sing_list['room_type']== 'Private room']
private_room.groupby('neighbourhood_cleansed') ['estimated_revenue_l365d'].sum().sort_values(ascending = False)


In [ ]:
#private room supply by location
private_room.groupby('neighbourhood_cleansed')['listing_id'].count().sort_values(ascending= False)

In [ ]:
#average private room price in Outram
293184/135

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Singapore River')
].groupby('price_range')['estimated_occupancy_l365d'].sum()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Singapore River')
].groupby('price_range')['estimated_revenue_l365d'].sum()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Singapore River')
]['host_id'].value_counts()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
]['price_range'].value_counts()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
]['price_range'].value_counts()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
]['host_id'].value_counts()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
].groupby('price_range')['estimated_occupancy_l365d'].mean()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
].groupby('price_range')['estimated_occupancy_l365d'].sum()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
].groupby('price_range')['estimated_revenue_l365d'].sum()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Geylang')
]['host_id'].count()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Outram')
].groupby('price_range')['estimated_occupancy_l365d'].sum()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Outram')
]['host_id'].value_counts()

In [ ]:
df_sing_list[
    (df_sing_list['room_type'] == 'Entire home/apt') &
    (df_sing_list['neighbourhood_cleansed'] == 'Outram')
].groupby('price_range')['estimated_revenue_l365d'].sum()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1. Get TOP 5 REVENUE + Marina South
revenue_data = (df_sing_list[df_sing_list['room_type'] == 'Entire home/apt']
                .groupby('neighbourhood_cleansed')['estimated_revenue_l365d']
                .sum()
                .sort_values(ascending=False)
                .head(5))

# Force include Marina South
if 'Marina South' in df_sing_list['neighbourhood_cleansed'].values:
    marina_south_revenue = (df_sing_list[(df_sing_list['room_type'] == 'Entire home/apt') &
                                        (df_sing_list['neighbourhood_cleansed'] == 'Marina South')]
                          ['estimated_revenue_l365d'].sum())
    revenue_data = pd.concat([revenue_data, pd.Series([marina_south_revenue], index=['Marina South'])])
    revenue_data = revenue_data.sort_values(ascending=False).head(6)  # Top 5 + Marina South

# 2. Filter supply data to match selected neighborhoods
supply_data = entire_home.groupby('neighbourhood_cleansed')['listing_id'].count()
supply_selected = supply_data.reindex(revenue_data.index)

# 3. Create double Y-axis chart
fig, ax1 = plt.subplots(figsize=(13, 9))

# X positions for bars
x = np.arange(len(revenue_data.index))
width = 0.35

# Left Y-axis: Supply (Number of Listings) - BLUE
bars1 = ax1.bar(x - width/2, supply_selected.values, width,
                label='Number of Listings',
                color='#1E90FF', alpha=0.85, edgecolor='#003087', linewidth=1.5)
ax1.set_xlabel('Neighborhood', fontweight='bold', fontsize=14)
ax1.set_ylabel('Number of Entire Home Listings', fontweight='bold', fontsize=13, color='#1E90FF')
ax1.tick_params(axis='y', labelcolor='#1E90FF')
ax1.set_xticks(x)
ax1.set_xticklabels(revenue_data.index, rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)

# Right Y-axis: Revenue - GREEN
ax2 = ax1.twinx()
bars2 = ax2.bar(x + width/2, revenue_data.values, width,
                label='Revenue (SGD)',
                color='#32CD32', alpha=0.85, edgecolor='#228B22', linewidth=1.5)
ax2.set_ylabel('Estimated Revenue (SGD)', fontweight='bold', fontsize=13, color='#32CD32')
ax2.tick_params(axis='y', labelcolor='#32CD32')

# Title and styling
plt.title('TOP 5 Revenue Neighborhoods + Marina South\nSupply vs Revenue Comparison (Entire Homes)',
          fontsize=18, fontweight='bold', pad=25, color='#000000')

# Add value labels on bars
max_supply = max(supply_selected.values) if len(supply_selected) > 0 else 1
max_revenue = max(revenue_data.values) if len(revenue_data) > 0 else 1

for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max_supply*0.02,
             f'{int(height):,}', ha='center', va='bottom',
             fontweight='bold', fontsize=11, color='#000000')

for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + max_revenue*0.02,
             f'${int(height):,}', ha='center', va='bottom',
             fontweight='bold', fontsize=11, color='#000000')

# Legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=11)

plt.tight_layout()

# Save for PowerPoint (optimized for #E9E9E9 background)
plt.savefig('top5_revenue_marina_south_comparison.png', dpi=300, bbox_inches='tight',
            facecolor='#E9E9E9', edgecolor='none')
plt.show()

# Print summary table
print("TOP 5 Revenue + Marina South Summary:")
summary_df = pd.DataFrame({
    'Neighborhood': revenue_data.index,
    'Revenue_SGD': revenue_data.values.round(0).astype(int),
    'Supply_Listings': supply_selected.values.astype(int)
})
print(summary_df)